In [8]:
search_for = 87
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0047838687896728516
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 88637461
type: [1, 1, 1, 1, 87] 87
cases of this type: 57289761
10000 15.019006635035932
20000 15.04451496294929
30000 48.46566430271516
40000 50.70862642670588
50000 46.685134627960224
60000 50.38123067186785
70000 48.496833890571295
80000 49.42918378625681
90000 49.59127564867681
100000 47.696998776264614
110000 47.35182299849199
120000 46.91457715402399
130000 49.131967899649226
140000 48.142266812537706
150000 46.70656843639266
160000 51.62054911389076
170000 55.07599531297795
180000 50.017550115283726
190000 48.43783549037158
200000 48.96280243382937
210000 47.238970372998466
220000 46.90358410194681
230000 46.75201568967991
240000 46.80278326570027
250000 47.78466802059186
260000 49.77219808398827
270000 45.676047048738916
280000 49.310488914233346
290000 47.29451071131818
300000 45.20817897141826
310000 57.53766193541830

3090000 39.94269062953171
3100000 44.17919755757105
3110000 43.52240147419765
3120000 44.56802680342699
3130000 44.42997296239042
3140000 44.798208934891925
3150000 45.71866863656382
3160000 41.869868868603426
3170000 44.84175695785702
3180000 39.67278595544363
3190000 39.51713925299047
3200000 45.445524421605775
3210000 40.24911607765668
3220000 42.71206019543302
3230000 43.81764115729262
3240000 48.31075487381508
3250000 17.72961818817679
3260000 17.310897163501554
3270000 17.65437684301838
3280000 28.55554799673001
3290000 54.338879061105175
3300000 45.61867798581262
3310000 47.91684402265078
3320000 35.7834785964517
3330000 14.598791355841739
3340000 14.492263463334101
3350000 14.393219943663665
3360000 40.115900970261166
3370000 48.44573943664924
3380000 41.07590356585796
3390000 43.02238299345048
3400000 42.291715909812574
3410000 42.01997718076942
3420000 40.208602494466426
3430000 40.148164496760096
3440000 46.1337118322159
3450000 41.65988556728117
3460000 41.904719046971515
3

6210000 38.1373451236385
6220000 35.26885420461942
6230000 39.827909335207266
6240000 34.47270394531209
6250000 7.259283311027368
6260000 7.288563176410967
6270000 7.2342512212685355
6280000 29.068554302517715
6290000 45.57403479515229
6300000 36.40980398329504
6310000 35.31721038866017
6320000 38.915814776845444
6330000 38.89993012562329
6340000 37.13563894553812
6350000 38.17623521446394
6360000 40.18194726825059
6370000 37.92109819139135
6380000 39.493022617225805
6390000 44.73643992125104
6400000 42.23375258953947
6410000 41.55174253415675
6420000 43.54731398462074
6430000 43.02198170426931
6440000 41.91605538688553
6450000 41.70850295151017
6460000 45.09733813928848
6470000 42.9031090587332
6480000 38.07838809882088
6490000 46.74937546386119
6500000 28.5853364322853
6510000 16.976981990127744
6520000 17.49619925322485
6530000 17.21948120353701
6540000 43.99743403033955
6550000 43.919616386296696
6560000 39.593030573580464
6570000 43.253911729558155
6580000 46.82386102005603
659000

9330000 13.323505021480376
9340000 13.458201283809542
9350000 20.66881758840436
9360000 41.23907208748159
9370000 35.83224760404471
9380000 37.40845259455838
9390000 37.339696537524226
9400000 36.37525367826992
9410000 37.24380248132763
9420000 35.40623096219741
9430000 36.47583591126281
9440000 38.704976632658074
9450000 35.885156312422545
9460000 34.08541275995572
9470000 34.256320191909076
9480000 31.74349620977585
9490000 32.54910466373841
9500000 34.78853887239288
9510000 33.168338542016
9520000 33.231469611468135
9530000 36.91724044185783
9540000 6.763230662281295
9550000 6.755325592989306
9560000 6.801218776364917
9570000 17.698726865834903
9580000 37.578567900336985
9590000 34.24852205588466
9600000 34.64898153610649
9610000 32.954361923558054
9620000 33.92972936605761
9630000 33.02516443386352
9640000 32.2244640262292
9650000 34.171964623319546
9660000 37.23414923929252
9670000 37.13307130916701
9680000 35.73860263628885
9690000 36.742253997397356
9700000 39.30507929950696
971

12370000 15.582821678956865
12380000 15.31090087239439
12390000 15.581562627356695
12400000 39.31338778556913
12410000 34.687762363569824
12420000 36.93318217107114
12430000 37.3951121366579
12440000 37.24842768172707
12450000 37.938661119735656
12460000 37.464377285662174
12470000 35.23031964841411
12480000 36.35052265925943
12490000 36.783115406630934
12500000 36.95899589884811
12510000 38.145014663030004
12520000 39.450513799513224
12530000 38.69344823425368
12540000 38.098237839688636
12550000 38.858850009820664
12560000 37.20394840177693
12570000 35.25543012865509
12580000 38.45504130264902
12590000 37.23873879795355
12600000 38.81495246421514
12610000 42.18012502221765
12620000 36.834358841084736
12630000 34.764255017239336
12640000 43.03112876099991
12650000 14.233233985745828
12660000 13.335638641112652
12670000 13.248452896948637
12680000 18.884613888696975
12690000 37.73301023147851
12700000 35.91606314665055
12710000 36.5790086049188
12720000 32.01654526634301
12730000 32.94

...

75180000 2.817661565240423
75190000 3.1330973095164896
75200000 2.7466939578949803
75210000 3.307935811185128
75220000 2.2857409269703033
75230000 3.260173259987434
75240000 2.690943615161783
75250000 3.2753876456980704
75260000 3.5115979712731047
75270000 2.7435212905992206
75280000 3.5125965314483043
75290000 2.0139532107013864
75300000 3.3815950457998776
75310000 2.2219594998922347
75320000 3.32975067551842
75330000 3.3801046319793597
75340000 3.359795803949674
75350000 1.7176817512299554
75360000 1.0955684318136243
75370000 1.1296804162391028
75380000 1.1319879305179847
75390000 1.099672676242464
75400000 1.0526422896992564
75410000 1.0003327265985873
75420000 1.0989128690565295
75430000 1.1041045253877442
75440000 1.156545867031455
75450000 1.0709529739826984
75460000 2.645651856457154
75470000 2.804416118831641
75480000 4.013499557842923
75490000 2.862862240039766
75500000 2.8878643578382794
75510000 3.26079131812042
75520000 2.142493999622027
75530000 3.442397712802569
75540000 

78150000 3.053719113416162
78160000 2.2810919399331806
78170000 3.038846925315559
78180000 2.300400981762515
78190000 2.761335022064388
78200000 2.936229294839826
78210000 2.1257933520924386
78220000 2.7424432463698585
78230000 1.94352677367621
78240000 2.670143825511025
78250000 2.934305858607451
78260000 2.139152165170504
78270000 2.6509480929193034
78280000 2.026892384999037
78290000 2.795205812894092
78300000 2.638412606539746
78310000 2.3021181355972486
78320000 2.9680976968409882
78330000 2.071926375748317
78340000 2.811861567579508
78350000 1.9277585581094292
78360000 2.295605632389254
best so far: 0
type: [1, 1, 1, 87, 1] 87
cases of this type: 658503
78370000 1.0141126609709858
78380000 0.5747432218175929
78390000 0.6237361275254819
78400000 0.6571966273366212
78410000 0.7047610216522548
78420000 0.6630736424658498
78430000 0.6789997868878047
78440000 0.6759899707873597
78450000 0.6831170283467173
78460000 0.7005688326712846
78470000 0.6938874021730423
78480000 0.6984546429077

81060000 1.5976973169310753
81070000 1.5847513623859881
81080000 1.6526905793091589
81090000 1.5816279093857608
81100000 1.5290021483886838
81110000 1.8727085597823727
81120000 1.7733547966353167
81130000 1.5292869047486384
81140000 1.51646231290657
81150000 1.6304979703122575
81160000 1.4221562145119906
81170000 1.6281717356663346
81180000 1.4905578229312766
81190000 1.4342990038885277
81200000 1.5476315137252143
81210000 1.6541855812221369
81220000 1.3490892600244284
81230000 1.6212439880549443
81240000 1.5551500464244417
81250000 1.4388846329674125
81260000 1.5660595366504193
81270000 1.635033494761732
81280000 1.5692159633026124
81290000 1.7264434222021234
81300000 1.8313549922752976
81310000 1.6125050221436619
81320000 1.6198230772915019
81330000 1.8238375852616298
81340000 1.6511957765530945
81350000 1.6898287340633207
81360000 1.8459165819280083
81370000 1.6404750813891094
81380000 1.472387764301505
81390000 1.7347227079681158
81400000 1.5862031296820838
81410000 1.4720541617256

84000000 0.9488432693954044
84010000 1.1096060132671595
84020000 1.054442280477345
84030000 0.9822033655769626
84040000 1.0637529098892213
84050000 0.9576153619215952
84060000 0.9060686565612288
84070000 1.0435367727688354
84080000 0.9692090097876788
84090000 0.9285325809174776
84100000 1.0086523965121508
84110000 1.0287354701009326
84120000 0.9375147927223642
84130000 0.9068278154643774
84140000 0.9926807821584014
84150000 0.8382295546173254
84160000 0.928527337321202
84170000 1.0424955697192615
84180000 0.9874073733210696
84190000 1.0674562772970397
84200000 0.9706218287080394
84210000 0.8826446080334582
84220000 0.8636537188993694
84230000 0.9713396429809398
84240000 0.9433994429236121
84250000 1.0319237713220915
84260000 1.0457877794464296
84270000 0.9195560577670799
84280000 0.8136655789153974
84290000 0.9398147132820089
84300000 0.8633453105523057
84310000 0.9124087270961405
84320000 1.0195674994287491
84330000 0.998937590390563
84340000 0.9385035009874105
84350000 0.865821658399

86800000 0.14210161630910634
86810000 0.12142892534818914
86820000 0.141583989726305
86830000 0.13022707767516375
86840000 0.13574178853426377
86850000 0.13262971457821793
86860000 0.13811007168916864
86870000 0.1358068747850193
86880000 0.12851322664161524
86890000 0.12341578808913628
86900000 0.13360276298812365
86910000 0.10848983079316882
86920000 0.12351400136727095
86930000 0.12173894431055916
86940000 0.12761568130950132
86950000 0.25785252931255104
86960000 0.6116303142182032
86970000 0.5907685376551483
86980000 0.29053391571086645
86990000 0.138764034704215
87000000 0.3312056217581299
87010000 0.5901150553486944
87020000 0.619077618758109
87030000 0.5122239638806118
87040000 0.4590746921585798
87050000 0.32051405106755754
87060000 0.15671403613358736
87070000 0.20888095076886812
87080000 0.41285204289745625
87090000 0.4365426455530458
87100000 0.4568131232163906
87110000 0.3766607531019913
87120000 0.2938749859790537
87130000 0.1725570316226482
87140000 0.16528954764023093
871